### Importar librerias

In [1]:
# keras module for building LSTM
import tensorflow as tf
from keras.utils import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
from sklearn.model_selection import train_test_split 
from tensorflow.keras.optimizers.schedules import ExponentialDecay
# set seeds for reproducability
from numpy.random import seed
tf.random.set_seed(42)
seed(42)

import pandas as pd
import numpy as np
import string, os
import matplotlib.pyplot as plt
import pickle
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

2023-07-12 17:05:04.069676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-12 17:05:04.126912: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-12 17:05:04.468918: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-07-12 17:05:04.468947: W tensorflow/compiler/xl

### Importar el dataset

In [3]:
df_recipe_dataset = pd.read_csv("/workspace/recipe_nlg_1000_new.csv",sep=';')
replacement=[]
for row in range(len(df_recipe_dataset["Full Recipe"])):
  string = df_recipe_dataset["Full Recipe"][row].replace("<|startofrecipe|>","")
  string = string.replace("\n"," ")
  replacement.append(string.replace("<|endofrecipe|>",""))
df_recipe_dataset["Full Recipe"] = replacement
display(df_recipe_dataset.head())


,Unnamed: 0,Full Recipe
0,0,ingredients: brown sugar milk vanilla nuts but...
1,1,ingredients: beef chicken breasts cream of mus...
2,2,ingredients: frozen corn cream cheese butter g...
3,3,ingredients: chicken chicken gravy cream of mu...
4,4,ingredients: peanut butter graham cracker crum...


In [5]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    with open('/workspace/tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    ## convert data to sequence of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words
corpus=df_recipe_dataset["Full Recipe"]
inp_sequences, total_words = get_sequence_of_tokens(df_recipe_dataset["Full Recipe"])
inp_sequences[:10]

[[3, 45],
 [3, 45, 5],
 [3, 45, 5, 28],
 [3, 45, 5, 28, 39],
 [3, 45, 5, 28, 39, 70],
 [3, 45, 5, 28, 39, 70, 20],
 [3, 45, 5, 28, 39, 70, 20, 468],
 [3, 45, 5, 28, 39, 70, 20, 468, 328],
 [3, 45, 5, 28, 39, 70, 20, 468, 328, 402],
 [3, 45, 5, 28, 39, 70, 20, 468, 328, 402, 124]]

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()

    # Add Input Embedding Layer
    model.add(Embedding(total_words, 16, input_length=input_len))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(512, recurrent_dropout=0.5, return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(256, recurrent_dropout=0.25,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    
    
    initial_learning_rate = 0.001 
    lr_schedule = ExponentialDecay( 
    initial_learning_rate, 
    decay_steps=800, 
    decay_rate=0.5, 
    staircase=True) #decreciendo en función de escalera 
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule), metrics=['acc'])

    return model

model = create_model(max_sequence_len, total_words)
model.summary()

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(predictors, label, test_size = 0.2, random_state=42)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3, start_from_epoch=10)]
batch_size=64
STEP_SIZE_TRAIN=0.8*X_train.shape[0]//batch_size
STEP_SIZE_VALID=0.2*X_val.shape[0]//batch_size
history = model.fit(X_train, 
                    y_train,
                    steps_per_epoch=STEP_SIZE_TRAIN,  
                    validation_data=(X_val,y_val), 
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20, 
                    batch_size=batch_size, 
                    verbose=1, 
                    callbacks=callbacks)

In [ ]:
model.save("/workspace/scratch_3.h5")
model.save_weights("/workspace/scratch_3")

In [ ]:
def plot_training_curves(H):
    # Muestro gráfica de accuracy y losses
    fig = plt.figure(figsize=(10,5))
    plt.style.use("ggplot")
    plt.subplot(1,2,1)
    plt.plot(np.arange(0, len(H.history['loss'])), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, len(H.history['loss'])), H.history["val_loss"], label="val_loss")
    plt.title('Training loss')
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(np.arange(0, len(H.history['loss'])), H.history["acc"], label="train_acc")
    plt.plot(np.arange(0, len(H.history['loss'])), H.history["val_acc"], label="val_acc")
    plt.title('Training Accuracy')
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.savefig("/workspace/performance_scratch_3.png")

In [ ]:
plot_training_curves(history)

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_x = model.predict(token_list, verbose=0)
        predicted=np.argmax(predicted_x,axis=1)
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
print (generate_text("ingredients: butter milk sugar", 100, model, max_sequence_len))